In [11]:
import numpy as np
import pandas as pd
import os
import glob
import cv2
import mediapipe as mp

In [10]:
def count_subdirs_and_images(root_dir):
    total_subdirs = 0
    total_images = 0

    # Iterate over all subdirectories with in the roor directory.
    for subdir, dirs, _ in os.walk(root_dir):
        for d in dirs:
            # Increment the subdirectory count.
            total_subdirs += 1

            # Count the number of images in the subdirectory.
            image_count = len(glob.glob(os.path.join(subdir, d, '*.*')))
            total_images += image_count
            print(f"Subdirectory: {os.path.join(subdir, d)}, Images: {image_count}")

    return total_subdirs, total_images


total_subdirs, total_images = count_subdirs_and_images("../dataset")
print(f"Total subdirectories: {total_subdirs}")
print(f"Total images: {total_images}")

Subdirectory: ../dataset/ashtanga namaskara, Images: 34
Subdirectory: ../dataset/chakravakasana, Images: 70
Subdirectory: ../dataset/eka pada rajakapotasana ii, Images: 55
Subdirectory: ../dataset/anantasana, Images: 43
Subdirectory: ../dataset/kapotasana, Images: 57
Subdirectory: ../dataset/hanumanasana, Images: 35
Subdirectory: ../dataset/parighasana, Images: 43
Subdirectory: ../dataset/marichyasana i, Images: 49
Subdirectory: ../dataset/tittibhasana, Images: 57
Subdirectory: ../dataset/janu sirsasana, Images: 48
Subdirectory: ../dataset/yoganidrasana, Images: 46
Subdirectory: ../dataset/kurmasana, Images: 40
Subdirectory: ../dataset/ananda balasana, Images: 59
Subdirectory: ../dataset/krounchasana, Images: 45
Subdirectory: ../dataset/phalakasana, Images: 57
Subdirectory: ../dataset/garbha pindasana, Images: 36
Subdirectory: ../dataset/adho mukha svanasana, Images: 69
Subdirectory: ../dataset/eka pada koundinyanasana i, Images: 51
Subdirectory: ../dataset/vajrasana, Images: 54
Subdir

In [14]:
# Initialize the MediaPipe pose.
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=True)

# Load Image.
image_path = '/Users/jyothivishnuvardhankolla/Desktop/Yoga_pose_estimation/dataset/adho mukha svanasana/1. 5-benefits-of-downward-facing-dog-pose.png'
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Perform pose estimation.
result = pose.process(image_rgb)

# Draw keypoints and connections on the image
annotated_image = image.copy()
mp_drawing.draw_landmarks(annotated_image, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

# Save the annotated image
cv2.imwrite('annotated_image.jpg', annotated_image)

# Print keypoints
if result.pose_landmarks:
    for idx, landmark in enumerate(result.pose_landmarks.landmark):
        print(f"Keypoint {idx}: ({landmark.x}, {landmark.y}, {landmark.z})")

# Release resources
pose.close()


Keypoint 0: (0.40662992000579834, 0.6675918102264404, -0.15713515877723694)
Keypoint 1: (0.3903322219848633, 0.6609479784965515, -0.20654842257499695)
Keypoint 2: (0.39113500714302063, 0.6561645865440369, -0.20647476613521576)
Keypoint 3: (0.392063707113266, 0.6511662602424622, -0.20650988817214966)
Keypoint 4: (0.3863912522792816, 0.6602478623390198, -0.1542823165655136)
Keypoint 5: (0.3841269612312317, 0.6550816297531128, -0.1542147994041443)
Keypoint 6: (0.38203302025794983, 0.6501891613006592, -0.15405023097991943)
Keypoint 7: (0.3943302035331726, 0.6016361117362976, -0.3573049008846283)
Keypoint 8: (0.38292255997657776, 0.6005789637565613, -0.11064118891954422)
Keypoint 9: (0.4238530099391937, 0.6358492374420166, -0.20126274228096008)
Keypoint 10: (0.4182398021221161, 0.6342649459838867, -0.12993477284908295)
Keypoint 11: (0.45538991689682007, 0.5190209746360779, -0.41984823346138)
Keypoint 12: (0.3981609642505646, 0.47277945280075073, -0.003718205261975527)
Keypoint 13: (0.361206